In [ ]:
import numpy as np
import pandas as pd
import itertools
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import classification_report
import re
import string 
from sklearn.metrics import accuracy_score, confusion_matrix


In [ ]:
df_fake=pd.read_csv('/content/Fake.csv',engine='python',error_bad_lines=False)
df_true=pd.read_csv('/content/True.csv',engine='python',error_bad_lines=False)

Skipping line 1606: unexpected end of data
Skipping line 1619: unexpected end of data


In [ ]:
df_fake.head(10)

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017"
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017"
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017"
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017"
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017"


In [ ]:
df_true.head(10)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"
5,"White House, Congress prepare for talks on spe...","WEST PALM BEACH, Fla./WASHINGTON (Reuters) - T...",politicsNews,"December 29, 2017"
6,"Trump says Russia probe will be fair, but time...","WEST PALM BEACH, Fla (Reuters) - President Don...",politicsNews,"December 29, 2017"
7,Factbox: Trump on Twitter (Dec 29) - Approval ...,The following statements were posted to the ve...,politicsNews,"December 29, 2017"
8,Trump on Twitter (Dec 28) - Global Warming,The following statements were posted to the ve...,politicsNews,"December 29, 2017"
9,Alabama official to certify Senator-elect Jone...,WASHINGTON (Reuters) - Alabama Secretary of St...,politicsNews,"December 28, 2017"


In [ ]:
df_fake.shape,df_true.shape

((1604, 4), (1617, 4))

In [ ]:
df_fake["class"]=0
df_true["class"]=1

In [ ]:
df_fake_manual_testing=df_fake.tail(10)
for i in range(402,392,-1):
    df_fake.drop([i],axis=0,inplace=True)
df_true_manual_testing=df_true.tail(10)
for i in range(404,394,-1):
    df_true.drop([i],axis=0,inplace=True)

In [ ]:
df_manual_testing=pd.concat([df_fake_manual_testing,df_true_manual_testing],axis=0)
df_manual_testing.to_csv("Manual_testing.csv")

In [ ]:
df_merge=pd.concat([df_fake,df_true],axis=0)
df_merge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [ ]:
df=df_merge.drop(["title","subject","date"],axis=1)
df.head(10)

,text,class
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0
5,The number of cases of cops brutalizing and ki...,0
6,Donald Trump spent a good portion of his day a...,0
7,In the wake of yet another court decision that...,0
8,Many people have raised the alarm regarding th...,0
9,Just when you might have thought we d get a br...,0


In [ ]:
df=df.sample(frac=1)
df.head(10)

,text,class
1245,The apple truly doesn t fall far from the tree...,0
519,"Ron Paul, who s a retired doctor, not an econo...",0
745,WASHINGTON (Reuters) - The Congressional Budge...,1
1230,Former FBI Director James Comey is testifying ...,0
346,"(Reuters) - Richard Cordray, a Democrat whose ...",1
3,WASHINGTON (Reuters) - Trump campaign adviser ...,1
468,"Donald Trump has the maturity of a toddler, an...",0
234,"Trump, who keeps promising to either scrap our...",0
526,This week s chaotic news cycle was mostly domi...,0
976,WASHINGTON (Reuters) - President Donald Trump’...,1


In [ ]:
df.isnull().sum()

text     0
class    0
dtype: int64

In [ ]:
def word_drop(text):
  text=text.lower()
  text=re.sub('\[.*?\]','',text)
  text=re.sub("\\W"," ",text)
  text=re.sub('https?://\S+|www\.\S+', '',text)
  text=re.sub('<.*?>+', '',text)
  text=re.sub('[%s]' % re.escape(string.punctuation),'',text)
  text=re.sub('\n','',text)
  text=re.sub('\w*\d\w*','',text)
  return text

In [ ]:
df["text"]=df["text"].apply(word_drop)

In [ ]:
df.head(10)

,text,class
1245,the apple truly doesn t fall far from the tree...,0
519,ron paul who s a retired doctor not an econo...,0
745,washington reuters the congressional budge...,1
1230,former fbi director james comey is testifying ...,0
346,reuters richard cordray a democrat whose ...,1
3,washington reuters trump campaign adviser ...,1
468,donald trump has the maturity of a toddler an...,0
234,trump who keeps promising to either scrap our...,0
526,this week s chaotic news cycle was mostly domi...,0
976,washington reuters president donald trump ...,1


In [ ]:
x=df["text"]
y=df["class"]

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.25)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorization=TfidfVectorizer()
xv_train=vectorization.fit_transform(x_train)
xv_test=vectorization.transform(x_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RFC=RandomForestClassifier()
RFC.fit(xv_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
RFC.score(xv_test,y_test)

0.9937578027465668

In [ ]:
pred_RFC = RFC.predict(xv_test)

In [ ]:
print(classification_report(y_test,pred_RFC))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       416
           1       1.00      0.99      0.99       385

    accuracy                           0.99       801
   macro avg       0.99      0.99      0.99       801
weighted avg       0.99      0.99      0.99       801



In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
LR=LogisticRegression()
LR.fit(xv_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
LR.score(xv_test,y_test)

0.9900124843945068

In [ ]:
pred_LR=LR.predict(xv_test)

In [ ]:
print(classification_report(y_test,pred_LR))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       416
           1       0.99      0.99      0.99       385

    accuracy                           0.99       801
   macro avg       0.99      0.99      0.99       801
weighted avg       0.99      0.99      0.99       801



In [ ]:
def output_lable(n):
  if n == 0:
    return "Fake news"
  elif n== 1:
    return "Not a fake news"
def manual_testing(news):
  testing_news={"text":[news]}
  new_def_test=pd.DataFrame(testing_news)
  new_def_test["text"]=new_def_test["text"]
  new_x_test=new_def_test["text"]
  new_xv_test=vectorization.transform(new_x_test)
  pred_RFC=RFC.predict(new_xv_test)
  pred_LR=LR.predict(new_xv_test)

  return print("\n RFC Prediction: {}\n LR Prediction: {}".format(output_lable(pred_RFC),(output_lable(pred_LR))))

In [ ]:
news=str(input())
manual_testing(news)

WASHINGTON (Reuters) - U.S. President Donald Trump and South Korean President Moon Jae-in agreed to strengthen their defense posture amid rising tensions with North Korea, the White House said on Friday. Trump and Moon, who met on Thursday, committed to strengthen their “combined defense posture, including through South Korea’s acquisition and development of highly advanced military assets” and “agreed to the enhanced deployment of U.S. strategic assets in and around South Korea on a rotational basis,” the White House said in a statement.

 RFC Prediction: Not a fake news
 LR Prediction: Not a fake news


In [ ]:
news=str(input())
manual_testing(news)

Massachusetts Rep. Jim McGovern brought the House down with a scathing rebuke of the pathetic Republican healthcare bill on Tuesday.He began by pointing out that the bill destroys all of the good things that the Affordable Care Act provides, such as covering preexisting conditions and allowing kids to stay on their parents insurance until the age of 26.McGovern conceded that the Affordable Care Act, also known as Obamacare, is not perfect. It has some issues that can be fixed, and Democrats have repeatedly offered to work with Republicans to fix the problems and cover more people while also bringing down costs.But as McGovern noted, Republicans are only obsessed with repealing the law. They have no interest in fixing the minor problems, many of which they created themselves by chipping away at the law over the years.So far, the Republican bill touted by Trump has failed several times in the House because even many Republicans are concerned about the fact that repealing the Affordable C